# "Net2Net: ACCELERATING LEARNING VIA KNOWLEDGE TRANSFER" paper implementation - https://arxiv.org/pdf/1511.05641.pdf

In [18]:
import torch as th
import numpy as np
from collections import Counter


def net_to_wider_net(m1, m2, new_width, bnorm=None, out_size=None, noise=True,
          random_init=True, weight_norm=True):
    w1 = m1.weight.data
    w2 = m2.weight.data
    b1 = m1.bias.data

    if "Conv" in m1.__class__.__name__ or "Linear" in m1.__class__.__name__:
        if "Conv" in m1.__class__.__name__ and "Linear" in m2.__class__.__name__:
            assert w2.size(1) % w1.size(
                0) == 0, "Linear units need to be multiple"
            if w1.dim() == 4:
                factor = int(np.sqrt(w2.size(1) // w1.size(0)))
                w2 = w2.view(w2.size(0), w2.size(1)//factor**2, factor, factor)
            elif w1.dim() == 5:
                assert out_size is not None, "For conv3d -> linear out_size is necessary"
                factor = out_size[0] * out_size[1] * out_size[2]
                w2 = w2.view(w2.size(0), w2.size(1)//factor, out_size[0],
                             out_size[1], out_size[2])

        old_width = w1.size(0)
        nw1 = m1.weight.data.clone()
        nw2 = w2.clone()

        if nw1.dim() == 4:
            nw1.resize_(new_width, nw1.size(1), nw1.size(2), nw1.size(3))
            nw2.resize_(nw2.size(0), new_width, nw2.size(2), nw2.size(3))
        elif nw1.dim() == 5:
            nw1.resize_(new_width, nw1.size(1), nw1.size(2),
                        nw1.size(3), nw1.size(4))
            nw2.resize_(nw2.size(0), new_width, nw2.size(2),
                        nw2.size(3), nw2.size(4))
        else:
            nw1.resize_(new_width, nw1.size(1))
            nw2.resize_(nw2.size(0), new_width)

        if b1 is not None:
            nb1 = m1.bias.data.clone()
            nb1.resize_(new_width)

        if bnorm is not None:
            nrunning_mean = bnorm.running_mean.clone().resize_(new_width)
            nrunning_var = bnorm.running_var.clone().resize_(new_width)
            if bnorm.affine:
                nweight = bnorm.weight.data.clone().resize_(new_width)
                nbias = bnorm.bias.data.clone().resize_(new_width)

        w2 = w2.transpose(0, 1)
        nw2 = nw2.transpose(0, 1)

        nw1.narrow(0, 0, old_width).copy_(w1)
        nw2.narrow(0, 0, old_width).copy_(w2)
        nb1.narrow(0, 0, old_width).copy_(b1)

        if bnorm is not None:
            nrunning_var.narrow(0, 0, old_width).copy_(bnorm.running_var)
            nrunning_mean.narrow(0, 0, old_width).copy_(bnorm.running_mean)
            if bnorm.affine:
                nweight.narrow(0, 0, old_width).copy_(bnorm.weight.data)
                nbias.narrow(0, 0, old_width).copy_(bnorm.bias.data)

        if weight_norm:
            for i in range(old_width):
                norm = w1.select(0, i).norm()
                w1.select(0, i).div_(norm)

        tracking = dict()
        for i in range(old_width, new_width):
            idx = np.random.randint(0, old_width)
            try:
                tracking[idx].append(i)
            except:
                tracking[idx] = [idx]
                tracking[idx].append(i)

            if random_init:
                n = m1.kernel_size[0] * m1.kernel_size[1] * m1.out_channels
                if m2.weight.dim() == 4:
                    n2 = m2.kernel_size[0] * \
                        m2.kernel_size[1] * m2.out_channels
                elif m2.weight.dim() == 5:
                    n2 = m2.kernel_size[0] * m2.kernel_size[1] * \
                        m2.kernel_size[2] * m2.out_channels
                elif m2.weight.dim() == 2:
                    n2 = m2.out_features * m2.in_features
                nw1.select(0, i).normal_(0, np.sqrt(2./n))
                nw2.select(0, i).normal_(0, np.sqrt(2./n2))
            else:
                nw1.select(0, i).copy_(w1.select(0, idx).clone())
                nw2.select(0, i).copy_(w2.select(0, idx).clone())
            nb1[i] = b1[idx]

        if bnorm is not None:
            nrunning_mean[i] = bnorm.running_mean[idx]
            nrunning_var[i] = bnorm.running_var[idx]
            if bnorm.affine:
                nweight[i] = bnorm.weight.data[idx]
                nbias[i] = bnorm.bias.data[idx]
            bnorm.num_features = new_width

        if not random_init:
            for idx, d in tracking.items():
                for item in d:
                    nw2[item].div_(len(d))

        w2.transpose_(0, 1)
        nw2.transpose_(0, 1)

        m1.out_channels = new_width
        m2.in_channels = new_width

        if noise:
            noise = np.random.normal(scale=5e-2 * nw1.std(),
                                     size=list(nw1.size()))
            nw1 += th.FloatTensor(noise).type_as(nw1)

        m1.weight.data = nw1

        if "Conv" in m1.__class__.__name__ and "Linear" in m2.__class__.__name__:
            if w1.dim() == 4:
                m2.weight.data = nw2.view(
                    m2.weight.size(0), new_width*factor**2)
                m2.in_features = new_width*factor**2
            elif w2.dim() == 5:
                m2.weight.data = nw2.view(m2.weight.size(0), new_width*factor)
                m2.in_features = new_width*factor
        else:
            m2.weight.data = nw2

        m1.bias.data = nb1

        if bnorm is not None:
            bnorm.running_var = nrunning_var
            bnorm.running_mean = nrunning_mean
            if bnorm.affine:
                bnorm.weight.data = nweight
                bnorm.bias.data = nbias

        return m1, m2, bnorm

In [9]:
def net_to_deeper_net(m, nonlin, bnorm_flag=False, weight_norm=True, noise=True):
    if "Linear" in m.__class__.__name__:
        m2 = th.nn.Linear(m.out_features, m.out_features)
        m2.weight.data.copy_(th.eye(m.out_features))
        m2.bias.data.zero_()

        if bnorm_flag:
            bnorm = th.nn.BatchNorm1d(m2.weight.size(1))
            bnorm.weight.data.fill_(1)
            bnorm.bias.data.fill_(0)
            bnorm.running_mean.fill_(0)
            bnorm.running_var.fill_(1)

    elif "Conv" in m.__class__.__name__:
        assert m.kernel_size[0] % 2 == 1, "Kernel size needs to be odd"

        if m.weight.dim() == 4:
            pad_h = int((m.kernel_size[0] - 1) / 2)
            m2 = th.nn.Conv2d(m.out_channels, m.out_channels,
                              kernel_size=m.kernel_size, padding=pad_h)
            m2.weight.data.zero_()
            c = m.kernel_size[0] // 2 + 1

        elif m.weight.dim() == 5:
            pad_hw = int((m.kernel_size[1] - 1) / 2)
            pad_d = int((m.kernel_size[0] - 1) / 2)
            m2 = th.nn.Conv3d(m.out_channels,
                              m.out_channels,
                              kernel_size=m.kernel_size,
                              padding=(pad_d, pad_hw, pad_hw))
            c_wh = m.kernel_size[1] // 2 + 1
            c_d = m.kernel_size[0] // 2 + 1

        restore = False
        if m2.weight.dim() == 2:
            restore = True
            m2.weight.data = m2.weight.data.view(m2.weight.size(0),
                                                 m2.in_channels,
                                                 m2.kernel_size[0],
                                                 m2.kernel_size[0])

        if weight_norm:
            for i in range(m.out_channels):
                weight = m.weight.data
                norm = weight.select(0, i).norm()
                weight.div_(norm)
                m.weight.data = weight

        for i in range(0, m.out_channels):
            if m.weight.dim() == 4:
                m2.weight.data.narrow(0, i, 1).narrow(
                    1, i, 1).narrow(2, c, 1).narrow(3, c, 1).fill_(1)
            elif m.weight.dim() == 5:
                m2.weight.data.narrow(0, i, 1).narrow(1, i, 1).narrow(
                    2, c_d, 1).narrow(3, c_wh, 1).narrow(4, c_wh, 1).fill_(1)

        if noise:
            noise = np.random.normal(scale=5e-2 * m2.weight.data.std(),
                                     size=list(m2.weight.size()))
            m2.weight.data += th.FloatTensor(noise).type_as(m2.weight.data)

        if restore:
            m2.weight.data = m2.weight.data.view(m2.weight.size(0),
                                                 m2.in_channels,
                                                 m2.kernel_size[0],
                                                 m2.kernel_size[0])

        m2.bias.data.zero_()

        if bnorm_flag:
            if m.weight.dim() == 4:
                bnorm = th.nn.BatchNorm2d(m2.out_channels)
            elif m.weight.dim() == 5:
                bnorm = th.nn.BatchNorm3d(m2.out_channels)
            bnorm.weight.data.fill_(1)
            bnorm.bias.data.fill_(0)
            bnorm.running_mean.fill_(0)
            bnorm.running_var.fill_(1)

    else:
        raise RuntimeError(
            "{} Module not supported".format(m.__class__.__name__))

    s = th.nn.Sequential()
    s.add_module('conv', m)
    if bnorm_flag:
        s.add_module('bnorm', bnorm)
    if nonlin is not None:
        s.add_module('nonlin', nonlin())
    s.add_module('conv_new', m2)

    return s

In [19]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import copy

batch_size = 64
test_batch_size = 1000
epochs = 10
lr = 0.01
momentum = 0.5
seed = 1
log_interval = 100

torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, x.size(1)*x.size(2)*x.size(3))
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

    def net2net_wider(self):
        self.conv1, self.conv2, _ = net_to_wider_net(self.conv1, self.conv2, 15)
        self.conv2, self.fc1, _ = net_to_wider_net(self.conv2, self.fc1, 30)
        print(self)

    def net2net_deeper(self):
        s = net_to_deeper_net(self.conv1, nn.ReLU, bnorm_flag=False)
        self.conv1 = s
        s = net_to_deeper_net(self.conv2, nn.ReLU, bnorm_flag=False)
        self.conv2 = s
        print(self)

    def define_wider(self):
        self.conv1 = nn.Conv2d(1, 15, kernel_size=5)
        self.conv2 = nn.Conv2d(15, 30, kernel_size=5)
        self.fc1 = nn.Linear(480, 50)

    def define_wider_deeper(self):
        self.conv1 = nn.Sequential(nn.Conv2d(1, 15, kernel_size=5),
                                      nn.ReLU(),
                                      nn.Conv2d(15, 15, kernel_size=5, padding=2))
        self.conv2 = nn.Sequential(nn.Conv2d(15, 30, kernel_size=5),
                                      nn.ReLU(),
                                      nn.Conv2d(30, 30, kernel_size=5, padding=2))
        self.fc1 = nn.Linear(480, 50)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data), Variable(target)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
        return 100. * correct / len(test_loader.dataset)

In [11]:
print("\n\n > Teacher training... ")
for epoch in range(1, epochs + 1):
    train(epoch)
    teacher_accu = test()



 > Teacher training... 
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.371851


/tmp/ipykernel_1166/2283106900.py:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.066881
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.206788
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.744905
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.599226
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.800742
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.752787
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.515826
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.474530
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.466711

Test set: Average loss: 0.2099, Accuracy: 9407/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.369939
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.456399
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.384596
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.451844
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.402165
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.289913
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.308951
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.618315
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.340878
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.297

In [20]:
print("\n\n > Wider Student training... ")
model_ = Net()
model_ = copy.deepcopy(model)

del model
model = model_
model.net2net_wider()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
for epoch in range(1, epochs + 1):
    train(epoch)
    wider_accu = test()



 > Wider Student training... 
Net(
  (conv1): Conv2d(1, 15, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(15, 30, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=480, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.387631


/tmp/ipykernel_1166/2545197502.py:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.621406
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.823580
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.691162
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.574728
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.593143
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.420739
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.445403
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.487250
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.364731


/home/petko/projects/ai-notebooks/env/lib/python3.11/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1749, Accuracy: 9476/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.422646
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.199908
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.270411
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.560420
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.458246
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.345032
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.355656
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.254552
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.434722
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.304286

Test set: Average loss: 0.1047, Accuracy: 9676/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.161064
Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.091402
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.286445
Train Epoch: 3 [19200/60000 (32%)]	Loss: 0.275425
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.214435
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.115487
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.322100
Train Epoch: 3 [44800/60000 (75%)]	Loss:

In [21]:
print("\n\n > Wider + Deeper Student training... ")
model_ = Net()
model_ = copy.deepcopy(model)

del model
model = model_
model.net2net_deeper()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
for epoch in range(1, epochs + 1):
    train(epoch)
    deeper_accu = test()



 > Wider + Deeper Student training... 
Net(
  (conv1): Sequential(
    (conv): Conv2d(1, 15, kernel_size=(5, 5), stride=(1, 1))
    (nonlin): ReLU()
    (conv_new): Conv2d(15, 15, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (conv2): Sequential(
    (conv): Conv2d(15, 30, kernel_size=(5, 5), stride=(1, 1))
    (nonlin): ReLU()
    (conv_new): Conv2d(30, 30, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=480, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 3.048279


/tmp/ipykernel_1166/2545197502.py:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.184439
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.259213
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.150717
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.067047
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.214770
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.321404
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.140248
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.115955
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.203985

Test set: Average loss: 0.0523, Accuracy: 9841/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.140832
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.273180
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.105051
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.048267
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.091193
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.218311
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.092276
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.077792
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.174223
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.114

In [22]:
print("\n\n > Wider teacher training... ")
model_ = Net()

del model
model = model_
model.define_wider()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
for epoch in range(1, 2*(epochs) + 1):
    train(epoch)
    wider_teacher_accu = test()



 > Wider teacher training... 
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.323987


/tmp/ipykernel_1166/2545197502.py:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.883135
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.049915
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.857313
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.531759
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.570595
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.533823
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.395554
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.504689
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.499775

Test set: Average loss: 0.1597, Accuracy: 9524/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.439629
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.228204
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.381115
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.218179
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.246535
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.282480
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.352200
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.278183
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.391891
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.357

In [23]:
print("\n\n > Wider + Deeper teacher training... ")
model_ = Net()

del model
model = model_
model.define_wider_deeper()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
for epoch in range(1, 3*(epochs) + 1):
    train(epoch)
    wider_deeper_teacher_accu = test()



 > Wider + Deeper teacher training... 
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305964


/tmp/ipykernel_1166/2545197502.py:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.285769
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.187154
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.188742
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.894444
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.479578
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.529480
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.424298
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.396830
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.349261

Test set: Average loss: 0.1603, Accuracy: 9528/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.319840
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.224067
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.372141
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.240714
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.224763
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.325720
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.345110
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.345676
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.287336
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.337

In [24]:
print(" -> Teacher:\t{}".format(teacher_accu))
print(" -> Wider model:\t{}".format(wider_accu))
print(" -> Deeper-Wider model:\t{}".format(deeper_accu))
print(" -> Wider teacher:\t{}".format(wider_teacher_accu))
print(" -> Deeper-Wider teacher:\t{}".format(wider_deeper_teacher_accu))

 -> Teacher:	98.44999694824219
 -> Wider model:	98.4800033569336
 -> Deeper-Wider model:	99.12999725341797
 -> Wider teacher:	99.02999877929688
 -> Deeper-Wider teacher:	99.4000015258789
